In [12]:
import json
import os
from pathlib import Path
import shutil

import numpy as np
import requests
import skimage.io
import yaml

%cd /home/ubuntu/hidebound/python
from hidebound.core.database import Database

import warnings
warnings.filterwarnings('ignore')

/home/ubuntu/hidebound/python


In [20]:
def setup_files(root='/tmp/mnt'):
    if Path(root).exists():
        shutil.rmtree(root)
    
    ingress = Path(f'{root}/ingress').as_posix()
    staging = Path(f'{root}/hidebound').as_posix()
    archive = Path(f'{root}/archive').as_posix()
    conf_dir = Path(f'{root}/hidebound/config').as_posix()
    os.makedirs(ingress)
    os.makedirs(staging)
    os.makedirs(archive)
    os.makedirs(conf_dir)
    
    config = dict(
        exclude_regex='\.DS_Store',
        staging_directory=staging,
        include_regex='',
        ingress_directory=ingress,
        specification_files=['/home/ubuntu/hidebound/python/hidebound/core/test_specifications.py'],
        write_mode='copy',
    )
    config_path = Path(conf_dir, 'hidebound_config.yaml')
    with open(config_path, 'w') as f:
        yaml.safe_dump(config, f)

    filepaths = [
        'p-cat001/spec001/p-cat001_s-spec001_d-running-cat_v001/p-cat001_s-spec001_d-running-cat_v001_c0000-0005_f0001.png',
        'p-cat001/spec001/p-cat001_s-spec001_d-running-cat_v001/p-cat001_s-spec001_d-Running-Cat_v001_c0000-0005_f0002.png',
#         'p-cat001/spec001/p-cat001_s-spec001_d-running-cat_v001/p-cat001_s-spec001_d-running-cat_v001_c0000-0005_f0002.png',
        'p-cat001/spec001/p-cat001_s-spec001_d-running-cat_v001/p-cat001_s-spec001_d-running-cat_v001_c0000-0005_f0003.png',

        'p-cat001/s-spec002/d-tabby-playing/p-cat001_s-spec002_d-tabby-playing_v001/p-cat001_s-spec002_d-tabby-playing_v001_f0001.jpg',
        'p-cat001/s-spec002/d-tabby-playing/p-cat001_s-spec002_d-tabby-playing_v001/p-cat001_s-spec002_d-tabby-playing_v001_f0002.jpg',
        'p-cat001/s-spec002/d-tabby-playing/p-cat001_s-spec002_d-tabby-playing_v001/p-cat001_s-spec002_d-tabby-playing_v001_f0003.jpg',

        'p-cat001/s-spec002/d-tabby-playing/p-cat001_s-spec002_d-tabby-playing_v002/p-cat001_s-spec002_d-tabby-playing_v002_f0001.jpg',
        'p-cat001/s-spec002/d-tabby-playing/p-cat001_s-spec002_d-tabby-playing_v002/p-cat001_s-spec002_d-tabby-playing_v002_f0002.jpg',
        'p-cat001/s-spec002/d-tabby-playing/p-cat001_s-spec002_d-tabby-playing_v002/p-cat001_s-spec002_d-tabby-playing_v002_f0003.jpg',

        'p-cat001/s-spec002/d-calico-jumping/p-cat001_s-spec002_d-calico-jumping_v001/p-cat001_s-spec002_d-calico-jumping_v001_f0001.jpg',
        'p-cat001/s-spec002/d-calico-jumping/p-cat001_s-spec002_d-calico-jumping_v001/p-cat001_s-spec002_d-calico-jumping_v001_f0002.jpg',
        'p-cat001/s-spec002/d-calico-jumping/p-cat001_s-spec002_d-calico-jumping_v001/p-cat001_s-spec002_d-calico-jumping_v001_f0003.jpg',
    ]
    filepaths = [Path(ingress, x) for x in filepaths]

    for filepath in filepaths:
        os.makedirs(filepath.parent, exist_ok=True)

        img = np.ones((1024, 1024, 3), dtype=np.uint8)
        img[:, :, 0] *= 255
        skimage.io.imsave(filepath.as_posix(), img)

setup_files()
!tree /tmp/mnt

/tmp/mnt
|-- archive
|-- hidebound
|   `-- config
|       `-- hidebound_config.yaml
`-- ingress
    `-- p-cat001
        |-- s-spec002
        |   |-- d-calico-jumping
        |   |   `-- p-cat001_s-spec002_d-calico-jumping_v001
        |   |       |-- p-cat001_s-spec002_d-calico-jumping_v001_f0001.jpg
        |   |       |-- p-cat001_s-spec002_d-calico-jumping_v001_f0002.jpg
        |   |       `-- p-cat001_s-spec002_d-calico-jumping_v001_f0003.jpg
        |   `-- d-tabby-playing
        |       |-- p-cat001_s-spec002_d-tabby-playing_v001
        |       |   |-- p-cat001_s-spec002_d-tabby-playing_v001_f0001.jpg
        |       |   |-- p-cat001_s-spec002_d-tabby-playing_v001_f0002.jpg
        |       |   `-- p-cat001_s-spec002_d-tabby-playing_v001_f0003.jpg
        |       `-- p-cat001_s-spec002_d-tabby-playing_v002
        |           |-- p-cat001_s-spec002_d-tabby-playing_v002_f0001.jpg
        |           |-- p-cat001_s-spec002_d-tabby-playing_v002_f0002.jpg
        |           `-- 

In [3]:
url = 'http://0.0.0.0:80/api/initialize'
config = dict(
    ingress_directory='/tmp/ingress',
    staging_directory='/tmp/hidebound',
    specification_files=['/home/ubuntu/hidebound/python/hidebound/core/test_specifications.py']
)
config = json.dumps(config)
response = requests.post(url, json=config).json()['message']
print(response)

url = 'http://0.0.0.0:80/api/update'
response = requests.post(url).json()['message']
print(response)

# url = 'http://0.0.0.0:80/api/read'
# response = requests.post(url).json()['response']

# url = 'http://0.0.0.0:80/api/search'
# params = json.dumps({'query': 'SELECT * FROM data WHERE asset_valid'})
# response = requests.post(url, json=params).json()['response']

url = 'http://0.0.0.0:80/api/create'
response = requests.post(url).json()['message']
print(response)

!tree /tmp/hidebound; tree /tmp/ingress

Database initialized.
Database updated.
Hidebound data created.
/tmp/hidebound
├── data
│   └── p-cat001
│       └── s-spec002
│           ├── d-calico-jumping
│           │   └── p-cat001_s-spec002_d-calico-jumping_v001
│           │       ├── p-cat001_s-spec002_d-calico-jumping_v001_f0001.jpg
│           │       ├── p-cat001_s-spec002_d-calico-jumping_v001_f0002.jpg
│           │       └── p-cat001_s-spec002_d-calico-jumping_v001_f0003.jpg
│           └── d-tabby-playing
│               ├── p-cat001_s-spec002_d-tabby-playing_v001
│               │   ├── p-cat001_s-spec002_d-tabby-playing_v001_f0001.jpg
│               │   ├── p-cat001_s-spec002_d-tabby-playing_v001_f0002.jpg
│               │   └── p-cat001_s-spec002_d-tabby-playing_v001_f0003.jpg
│               └── p-cat001_s-spec002_d-tabby-playing_v002
│                   ├── p-cat001_s-spec002_d-tabby-playing_v002_f0001.jpg
│                   ├── p-cat001_s-spec002_d-tabby-playing_v002_f0002.jpg
│                   └── p-cat001